In [7]:
from qiskit.primitives import Estimator
from scipy.optimize import minimize
from qc4pyscf.ansatz.UCC import UCC
from pyscf import gto,scf


In [8]:
###########################################
# Original PySCF Process: 
# 1. Make mol object
# 2. Change into RHF or UHF object
# 3. Run it to get MO
###########################################

# 1. Mol object
mol = gto.M(atom = "H 0.000000 0.000000 0; H 0.000000 0.000000 0.50000", 
            basis = "sto6g",
            charge = 0,
            spin = 0)

# 2 & 3. Make scf object and run
mf = scf.ROHF(mol).run() if mol.spin==0 else scf.ROHF(mol).run()


converged SCF energy = -1.05318793867456


In [9]:

###########################################
# Additional Process for Quantum Computer:
# 1. Put variables in UCC object as below
# 2. .Build() to make ansatz circuit & 2nd quantized Hamiltonian
# 3. Run minimizer with them above
###########################################

# 1. Make UCC object. Settings are same as Default
qc_mf = UCC(mol = mol,                                  # PySCF mol Object
            mf = mf,                                    # PySCF Object such as RHF or UHF
            ex_code = 'sd',                             # Excitation Code. s, d, t only
            mapping='jordan_wigner',                    # Mapping Method. Currently only jordan_wigner
            cd_acc = 1e-6)                              # Accuracy of Cholesky Decomposition while generating Hamiltonian

# 2. Build UCC Ingredients
qc_mf.build()


Alpha - alpha excitation denied
Beta - beta excitation denied
accuracy of Cholesky decomposition  9.205868230312285e-15
Ansatz & Hamiltonian Build Done


In [6]:
def cost_func_sim(params, ansatz, H, estimator):        # Cost Function: Simplify Estimating Process, Change If uses Real Backend
    res = estimator.run([ansatz], [H], params).result()
    energy = res.values[0]
    return energy

# 3. Minimizing with QC Simulator & SciPy Minimizer
x0 = [0.0] * qc_mf.ansatz.num_parameters                # Initial Parameter Vector of UCC
estimator = Estimator()                                 # Qiskit Estimator Simulator Object
res = minimize(cost_func_sim,                           # Minimizer
               x0, 
               args=(qc_mf.ansatz, qc_mf.H, estimator), 
               method="cobyla", 
               options={'maxiter': 10000}) 
energy = getattr(res,'fun')                             # Result Energy

print(f"Result Energy: {energy} Hartree")               # Ta-Da!


/tmp/ipykernel_2415215/2286501041.py:8: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()                                 # Qiskit Estimator Simulator Object


Result Energy: -1.0653851697501626 Hartree


In [ ]:
qc_mf.ansatz.draw('mpl')